Sjoerd Hermes

12-05-2020

Bayesian statistics

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from neupy import plots

In [ ]:
def PPCA(X, d, scaling = False):
    if d >= X.shape[1]:
        print("Please choose a number that is lower than the number of variables in the data.") 
    if type(d) != int:
        print("Please choose an integer value for d.") 
    scaler = StandardScaler(with_mean = True, with_std = False)
    if scaling == True:
        scaler = StandardScaler(with_mean = True, with_std = True)
    X_tilde = scaler.fit_transform(X)
    p = X_tilde.shape[1]
    S = np.cov(X_tilde.T, bias = True)    
    values, vectors = np.linalg.eig(S)
    order = sorted(range(len(values)), key = lambda k: values[k], reverse = True)
    V = np.zeros((p,p))
    for i in order:
        V[:, order.index(i)] = vectors[:,order[i]]
    V = V[:,:d]
    values = sorted(values, reverse = True)
    Lambda = np.diag(values[:d])
    sigma_2_hat = (1/(p-d))*sum(values[d:p])
    I = np.identity(d)
    comp = np.diag(np.sqrt(np.diag((Lambda - sigma_2_hat))))
    W_hat = np.dot(V, comp)
    M_hat = sigma_2_hat*I + np.dot(W_hat.T,W_hat) 
    post_mean = np.linalg.multi_dot([X_tilde, W_hat, np.linalg.inv(M_hat)])
    plt.figure(figsize=(16, 12))
    plots.hinton(W_hat)
    plt.show()
    return post_mean

# 1. PPCA
Principal component analysis (PCA) is a popular method which can be used when one is faced with a large set of correlated variables. Principal components allow us to summarize this set with a smaller number of representative variables that collectively explain most of the variability in the original set (James et al., 2009). This is done by removing correlated features, as the contribute little to the solution. Probabilistic principal component analysis is based of principal component analysis, but aims to solve one of the main disadvantages related to PCA: the absence of an associated probability density (Tipping and Bishop, 1999). Advantages of such a probability density are for example: allowing for principal component projection in the case of missing data, the possibility to make use of Bayesian inference methods by combining the likelihood with a prior and the extension of a single PCA model to a mixture of PCA models (Tipping and Bishop, 1999). As the mathematical derivation of PPCA differs from that of PCA, some explanation is required. This explanation follows the style of Tipping and Bishop (1999), but with the notation of the lecture slides, in order to be consistent with the assignment.

Suppose we have a $n \times p$ data matrix $\textbf{X}$, where $p$ represents the variables. $\textbf{X}$ is therefore $p$-dimensional. The aim of PCA, and PPCA is to reduce these $p$ dimensions into $d$ dimensions, where $d < p$. 

# 2. The code

The first step in the code contains a statement that informs the user whether a too high value for $d$ has been plugged in. After all, if $d \geq p$, it is not longer about dimensionality reduction, which is not in accordance with the aim of PPCA. Furthermore, as we will see later in this explanation, the term $\sigma^2$, denoting the average variance lost per discarded dimension has maximum likelihood estimate $\hat{\sigma}^2 = \frac{1}{p-d}\sum_{k = d+1}^{p}\lambda_{k}$. If $d = p$ we are dividing by 0, which is not possible. This explains why the value for $d$ has to be less than $p$. Next to that, the algorithm returns a statement saying that the user should use an integer value for $d$, if this has not been done, since the model van only work with integer values for $d$.

The second step contains a test whether or not the user wants to scale the data (this is done by setting <code>scaling = True</code> as an argument when running the code). If this is the case, the scaler variable gets the <code>with_std = True</code> instead of the <code>with_std = False</code>, which subsequently centers and scales the data matrix $\textbf{X}$ into $\tilde{\textbf{X}}$. If <code>with_std = True</code> is chosen instead of <code>with_std = False</code>, the covariance matrix of the next step $\frac{1}{n}\tilde{\textbf{X}}^{T}\tilde{\textbf{X}}$ is the correlation matrix instead. From here on out $\tilde{\textbf{X}}$ denotes both the centered data matrix and the centered and scaled data matrix, as the intuition remains the same throughout the rest of the explanation. Note that the data matrix $\tilde{\textbf{X}}$ is always centered, no matter which option for scaling is chosen. 

In the third step we define $p$, the number of variables in $\tilde{\textbf{X}}$, and compute the sample covariance matrix $\textbf{S}$. In the code, <code>S = np.cov(X_tilde.T, bias = True)</code>, two things stand out. The first one is that we use $\tilde{\textbf{X}}^{T}$ instead of $\tilde{\textbf{X}}$, and the second thing is that we set the bias to True. We use $\tilde{\textbf{X}}^{T}$, because this way, the sample covariance matrix is calculated in the way we want: $\frac{1}{n}\tilde{\textbf{X}}^{T}\tilde{\textbf{X}}$. As $\tilde{\textbf{X}}$ is a $n \times p$ matrix, where $p$ denotes the variables, we want $\textbf{S}$ to be a $p \times p$ matrix. Considering $\tilde{\textbf{X}}^{T}$ is a $p \times n$, matrix we therefore receive the desired $p \times p$ matrix $\tilde{\textbf{X}}^{T}\tilde{\textbf{X}}$. The bias was set to <code>True</code>, as this represents the sample covariance. The default is set to <code>False</code>, and thus has correction $\frac{1}{n-1}$ (Bessel's correction) instead of the desired $\frac{1}{n}$. Therefore, setting <code>np.cov(X_tilde.T, bias = True)</code> is equivalent to $\frac{1}{n}\tilde{\textbf{X}}^{T}\tilde{\textbf{X}}$. 

Continuing, we compute the eigenvalues and eigenvectors of $\textbf{S}$ using the <code>eig</code> function from  numpy's linear algebra library. These eigenvalues are then ordered by decreasing size, and this order is saved as a new variable <code>order</code>. Based on the order of the eigenvalues, matrix $\textbf{V}$ is constructed, which has the eigenvectors of $\textbf{S}$ as it's columns ordered based on <code>order</code> (decreasing order of the value of the eigenvalues, which is why <code>reverse = True</code> is used). This matrix contains $d$ eigenvectors corresponding to the largest eigenvalues. The the first column of $\textbf{V}$ contains the eigenvector corresponding to the largest eigenvalue, which is the direction along which the data set has the maximum variance. Each following column thus has an eigenvector corresponding to an eigenvalue which explains less of the variance than the previous column. This is because we computed the eigenvalues and eigenvectors of $\textbf{S}$, the covariance matrix. We arranged $\textbf{V}$ in this particular way, as the goal is to account for as much variance as possible, while keeping the dimensionality of the data to a minumum. Using the ordered eigenvalues, matrix $\boldsymbol\Lambda$ is a diagonal matrix containing the $d$ largest ordered eigenvalues, again using the <code>reverse = True</code> argument. All remaining eigenvalues, $\lambda_{d+1}...\lambda_{p}$ are used to calculate $\sigma^2$ where the maximum likelihood of $\sigma^2$ is given by: $\hat{\sigma}^2 = \frac{1}{p-d}\sum_{k = d+1}^{p}\lambda_{k}$, Tipping and Bishop (1999). This $\hat{\sigma}^2$ represents the average variance lost per discarded dimension. 

The next step consists of computing the term $\sqrt{\boldsymbol\Lambda_d - \hat{\sigma}^2\textbf{I}_d}$. This seems like a complex endavour, but remember that $\boldsymbol\Lambda_d$ is a diagonal matrix with the ordered eigenvalues $\lambda_k$ $k$ $\in$ {$1...d$}. We can the use the Spectral theorem on our symmetric positive definite matrix $\textbf{S}$, which allows a decomposition $\textbf{S} = \textbf{Q}\boldsymbol\Lambda \textbf{Q}^T$. We already did a decomposition of S in the previous step: <code>vector</code> is simply $\textbf{Q}$ and <code>values</code> are the diagonal elements of $\boldsymbol\Lambda$, such that <code>np.diag(values)</code> $ = \boldsymbol\Lambda$. By definition, its square root is $\textbf{S}^{\frac{1}{2}} = \textbf{Q}\boldsymbol\Lambda^{\frac{1}{2}} \textbf{Q}^T$, where $\boldsymbol\Lambda^{\frac{1}{2}}$ = $\textit{diag}$($\lambda^{\frac{1}{2}}$), as the square root of a diagonal matrix is simply the square root of its elements diagonalized. As $\hat{\sigma}^2\textbf{I}_d$ is simply $\textit{diag}(\hat{\sigma}^2)$ we can rewrite $\sqrt{\boldsymbol\Lambda_d - \hat{\sigma}^2\textbf{I}_d}$ as $\textit{diag}(\sqrt{\lambda_k - \hat{\sigma}^2})$.

We can now compute weight matrix $\textbf{W}$, using the maximum likelihood estimate as $\hat{\textbf{W}} = \textbf{V}_d(\boldsymbol\Lambda_d - \hat{\sigma}^2\textbf{I}_d)^{\frac{1}{2}}$. It should be noted that both the article of Tipping and Bishop (1999), as well as the lecture slides on PPCA, make use of a rotation matrix $\textbf{R}$ (for convenience setting this to identity matrix $\textbf{I}_{d}$ when computing matrix $\textbf{W}$, $\textbf{W} = \textbf{V}_d(\boldsymbol\Lambda_d - \hat{\sigma}^2\textbf{I}_d)^{\frac{1}{2}}\textbf{R}$.  However, this matrix $\textbf{R}$ was not mentioned in the assignment's formula for $\textbf{W}$. When testing results in the code with and without matrix $\textbf{R}$, the results were fully the same. This was tested for all combinations of centering and scaling the $\textbf{X}$ matrix. The difference between the assignment on one hand and the lecture slides and the article of Tipping and Bishop on the other is that we are working with the centered (and possibly scaled) matrix $\tilde{\textbf{X}}$ instead of $\textbf{X}$.

To compute the posterior means $\mathbb{E}[z_k|x_k], k \in$ {$1...n$}. We need matrix $\hat{\textbf{M}}$. 
From Tipping and Bishop (1999) we have that the posterior covariance matrix is given by $\sigma^2\textbf{M}^{-1} = \sigma^2(\sigma^2\textbf{I} + \textbf{W}^T\textbf{W})^{-1}$ 

$\implies \textbf{M}^{-1} = (\sigma^2\textbf{I} + \textbf{W}^T\textbf{W})^{-1}$ 

$\implies \textbf{M} = \sigma^2\textbf{I} + \textbf{W}^T\textbf{W}$

With this posterior covariance matrix, as Tipping and Bishop (1999) have shown, we can then simply compute the desired posterior means using
$\mathbb{E}[z_k|x_k] = \tilde{\textbf{X}}\hat{\textbf{W}}\hat{\textbf{M}}^{-1}$, this is slightly different from the formula used in the slides $\mathbb{E}[z_k|x_k] = \hat{\textbf{M}}^{-1}\hat{\textbf{W}}(x_k - \bar{x_n})$ this difference again stems from the fact that $\bar{x_n} = 0$ due to the centering.

# 3. PPCA in action

First an example using the Iris data set, where $p = 4$.

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
PPCA(X, 3, scaling = False)

Judging by the Hinton diagram, the first column has the strongest magnitude, but stating that the other dimensions are surpressed would go a bit far. As the dimensionality of the Iris data set is already small, lets try the PPCA algorithm on a data set with a higher dimensionality to see if the algorithm can sucesfully reduce the dimensionality of the data.

The second example consists of generated data $\textbf{X}$. Here $p$ is 20, which means that the data has more dimensions when compared to the previous example. 

In [ ]:
from sklearn.datasets import make_blobs
X = make_blobs(n_samples=500, centers = 3, n_features=20, shuffle = True, random_state=2020)[0]
PPCA(X, 19, scaling = False)

Here, we see a more clear distinction between the magnitudes of the elements for each dimension. Suppose we denote the number of centers $c$ and the number of features (dimensions) $f$, then according to the Hinton diagram that is outputted by the PPCA algorithm, the number of surpressed dimensions = $f - (c + 1)$. Once the value of $c$ gets too big relative to $f$, the algorithm has difficulty detecting the additional centers. 


# 4. References
James G., Witten D., Hastie T., and Tibshirani R. (2009). An introduction to statistical learning. Springer Series in Statistics. Springer-Verlag, New York.

Tipping, M. and Bishop, C., 1999. Mixtures of Probabilistic Principal Component Analyzers. Neural Computation, 11(2), pp.443-482.